# Используем Python для работы с базой данных PostgreSQL

<a target="_blank" href="https://colab.research.google.com/github/sozykin/middle_python/blob/main/05/05_postgresql.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Подключаем библиотеку psycopg2
import psycopg2

## Создаем соединение с базой данных

In [ ]:
connection = psycopg2.connect(
    database='postgres', # Название базы данных
    user='postgres',     # Имя пользователя
    password='1234qwer', # Пароль (не включайте пароли в продуктивный код!!!)
    host='localhost',    # Имя хоста сервера PostgreSQL
    port=5432,           # Порт, на котором работает сервер PostgreSQL
)

In [ ]:
# Создаем курсор
cursor = connection.cursor()

## Создание таблицы для данных

In [ ]:
# Оператор SQL для создания таблицы
create_names_table = """
CREATE TABLE IF NOT EXISTS names(
    id INTEGER,
    name TEXT,
    number_of_persons INTEGER,
    global_id INTEGER,
    year INTEGER,
    month TEXT 
)
"""

# Запускаем команду создания таблицы
cursor.execute(create_names_table)

# Фиксируем изменения
connection.commit()

## Вставка данных в таблицу

In [ ]:
# Оператор SQL для вставки данных в таблицу 
insert_data = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (1, 'Мария', 252, 37750254, 2015, 'январь')
"""

# Запускаем команду вставки данных
cursor.execute(insert_data)

# Фиксируем изменения
connection.commit()

## Запрос данных из таблицы

In [ ]:
# Запрос SQL для извлечения данных из таблицы names
select_data = "SELECT * FROM names"

# Запускаем запрос получения данных
cursor.execute(select_data)

# Получаем результаты запроса
resuls = cursor.fetchall()

# Печатаем результаты
for row in resuls:
    print(row)

## Завершение работы

In [ ]:
# Закрываем курсор
cursor.close()

In [ ]:
# Закрываем соединение
connection.close()

## Работа с транзакциями

In [ ]:
connection = psycopg2.connect(
    database='postgres', # Название базы данных
    user='postgres',     # Имя пользователя
    password='1234qwer', # Пароль (не включайте пароли в продуктивный код!!!)
    host='localhost',    # Имя хоста сервера PostgreSQL
    port=5432,           # Порт, на котором работает сервер PostgreSQL
)

In [ ]:
cursor = connection.cursor()

In [ ]:
delete_data = "DELETE FROM names" 

# Запускаем команду удаления данных
cursor.execute(delete_data)

In [ ]:
cursor.execute("SELECT * FROM names")

In [ ]:
cursor.fetchall()

In [ ]:
connection.rollback()

In [ ]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

Тестируем изоляцию транзакций

In [ ]:
delete_data = "DELETE FROM names" 

# Запускаем команду удаления данных
cursor.execute(delete_data)

In [ ]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

In [ ]:
# Оператор SQL для вставки данных в таблицу 
insert_several_rows = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (3, 'Анна', 190, 37750256, 2015, 'январь'),
       (4, 'Варвара', 190, 37750257, 2015, 'январь'),
       (5, 'Виктория', 186, 37750258, 2015, 'январь')
"""

In [ ]:
# Запускаем команду вставки данных
cursor.execute(insert_several_rows)

In [ ]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

In [ ]:
connection.commit()

## Работа с транзакциями через менеджер контекста

In [ ]:
insert_data = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (2, 'Анастасия', 224, 37750255, 2015, 'январь')
"""

In [ ]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(insert_data)

После блока with транзакция фиксируется (если не было исключений).

Соединение при этом не закрывается.

In [ ]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)


## Запросы с параметрами

In [ ]:
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%s, %s, %s, %s, %s, %s)
"""  

In [ ]:
# Значения для параметров запроса
rows = [
    (483, 'Амелия', 8, 62367755, 2015, 'май'),
    (484, 'Камила', 8, 62367756, 2015, 'май')
]

In [ ]:
with connection:
    with connection.cursor() as cursor:
        cursor.executemany(insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

Команда с именованными параметрами

In [ ]:
rows = [
    {'id': 483, 
     'name': 'Евангелина', 
     'number_of_persons': 8, 
     'global_id': 62367757, 
     'year': 2015, 
     'month': 'май'},
    {'id': 486, 
     'name': 'Альбина', 
     'number_of_persons': 8, 
     'global_id': 62367758, 
     'year': 2015, 
     'month': 'май'}
]

In [ ]:
# Подготавливаем запрос с именованными параметрами
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%(id)s, %(name)s, %(number_of_persons)s, %(global_id)s, %(year)s, %(month)s)
"""

In [ ]:
with connection:
    with connection.cursor() as cursor:
        # Запускаем команду вставки нескольких элементов данных
        cursor.executemany(insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

## Расширение psycopg2.extras

In [ ]:
from psycopg2.extras import execute_batch

In [ ]:
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%s, %s, %s, %s, %s, %s)
"""  

In [ ]:
# Значения для параметров запроса
rows = [
    (483, 'Амелия', 8, 62367755, 2015, 'май'),
    (484, 'Камила', 8, 62367756, 2015, 'май')
]

In [ ]:
with connection:
    with connection.cursor() as cursor:
        execute_batch(cursor, insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

## Загружаем данные из JSON в PostgreSQL

In [ ]:
import json

In [ ]:
with open("names_m.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
data

In [ ]:
len(data)

In [ ]:
# Оператор SQL для создания таблицы
create_names_table = """
CREATE TABLE IF NOT EXISTS names(
    id INTEGER,
    name TEXT,
    number_of_persons INTEGER,
    global_id BIGINT,
    year INTEGER,
    month TEXT 
)
"""

In [ ]:
# Подготавливаем запрос с именованными параметрами
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%(ID)s, %(Name)s, %(NumberOfPersons)s, %(global_id)s, %(Year)s, %(Month)s)
"""

In [ ]:
with connection:
    with connection.cursor() as cursor:
        # Удаляем таблицу
        cursor.execute("DROP TABLE IF EXISTS names")
        # Запускаем команду создания таблицы
        cursor.execute(create_names_table)

In [ ]:
with connection:
    with connection.cursor() as cursor:
        # Вставляем данные в таблицу
        execute_batch(cursor, insert_several_rows_parameters, data)

## Серверные курсоры

In [ ]:
with connection:
    with connection.cursor(name='select_names') as cursor:
        # Количество строк данных, получаемых за один раз
        cursor.itersize = 10
        cursor.execute("SELECT * FROM names")
        for row in cursor:
            print(row)